# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [14]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [15]:
weather_data = pd.read_csv("../WeatherPY/data/cities.csv")
weather_data.dropna()
weather_data.head()

,Unnamed: 0,City,Country,Date,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed
0,0,Hermanus,ZA,1650472100,-34.4187,19.2345,68.81,70,41,2.98
1,1,Clifton Springs,AU,1650472570,-38.1500,144.5667,57.94,88,75,4.61
2,2,At-Bashi,KG,1650472667,41.1702,75.8106,52.39,73,100,0.74
3,3,Kruisfontein,ZA,1650471913,-34.0033,24.7314,70.84,47,77,2.19
4,4,Atuona,PF,1650471847,-9.8000,-139.0333,78.85,73,59,20.42


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [16]:
#configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = weather_data[["Latitude", "Longitude"]]

# Store Humidity in humidity
humidity = weather_data["Humidity"]

In [17]:
# Plot the Heatmap
fig = gmaps.figure(center = [0,0] ,zoom_level = 2)

#Create and add heat layer 
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                               dissipating=False, max_intensity=100,
                               point_radius = 4)
fig.add_layer(heat_layer)
#Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [20]:
# Narrow down the cities with wind speed less than 10 mph, cloudiness equals to 0 and max temp between 60 and 80
narrowed_city_df = weather_data.loc[(weather_data["Wind Speed"] <= 10) & (weather_data["Cloudiness"] == 0) & \
                                   (weather_data["Max Temperature"] >= 70) & (weather_data["Max Temperature"] <= 80)].dropna()

narrowed_city_df

,Unnamed: 0,City,Country,Date,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed
19,19,Cape Town,ZA,1650472109,-33.9258,18.4232,75.88,41,0,3.44
84,84,Birjand,IR,1650472680,32.8663,59.2211,70.27,11,0,9.22
91,91,Port Elizabeth,ZA,1650472124,-33.9180,25.5701,70.32,79,0,6.91
136,136,Cidreira,BR,1650472271,-30.1811,-50.2056,71.87,64,0,7.23
161,161,Cabo San Lucas,MX,1650472186,22.8909,-109.9124,76.60,69,0,5.99
192,192,Sarakhs,IR,1650472703,36.5449,61.1577,71.74,53,0,0.00
403,403,Bucerías,MX,1650472754,20.7500,-105.3333,78.78,61,0,2.30
428,428,Pisco,PE,1650472761,-13.7000,-76.2167,71.65,73,0,8.05
516,516,Vao,NC,1650472784,-22.6667,167.4833,74.95,70,0,7.76
525,525,Jitotol,MX,1650472787,17.0333,-92.8667,76.42,28,0,1.90


### Hotel Map
* Create a dataframe named `hotel_df` that is the df created above only with the pertinent data.  See the prepared code below (`info_box_template`) to determine what data you will need for your final map.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels (aka "lodging") within 5000 meters.
* Utilize the Google Places API to find the first hotel search result.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [21]:
# Create dataframe and empty column for hotel names
hotel_df = pd.DataFrame(weather_data, columns=["City", "Country", "Latitude", "Longitude"])

hotel_df["Hotel Name"] = ""
# hotel_df["Coordinates"]=""
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
0,Hermanus,ZA,-34.4187,19.2345,
1,Clifton Springs,AU,-38.1500,144.5667,
2,At-Bashi,KG,41.1702,75.8106,
3,Kruisfontein,ZA,-34.0033,24.7314,
4,Atuona,PF,-9.8000,-139.0333,
...,...,...,...,...,...
564,Gushikawa,JP,26.3544,127.8686,
565,Crotone,IT,39.0851,17.1178,
566,Byron Bay,AU,-28.6500,153.6167,
567,Pacific Grove,US,36.6177,-121.9166,


In [22]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [23]:
for index, row in hotel_df.iterrows():
    # get city name, latitude, longitude from df
    lat = row["Latitude"]
    lng = row["Longitude"]
    city_name = row["City"]
    hotel_name = ["name"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    

# Print end of search once searching is completed
print("-------End of Search-------")

Retrieving Results for Index 0: Hermanus.
Closest hotel in Hermanus is Misty Waves Boutique Hotel.
------------
Retrieving Results for Index 1: Clifton Springs.
Closest hotel in Clifton Springs is Drysdale Hotel.
------------
Retrieving Results for Index 2: At-Bashi.
Closest hotel in At-Bashi is Hostel At-Bashy.
------------
Retrieving Results for Index 3: Kruisfontein.
Closest hotel in Kruisfontein is Humansdorp Boutique Hotel.
------------
Retrieving Results for Index 4: Atuona.
Closest hotel in Atuona is Hôtel Hiva Oa Hanakee Lodge.
------------
Retrieving Results for Index 5: Natal.
Closest hotel in Natal is Yak Beach Hotel.
------------
Retrieving Results for Index 6: Albany.
Missing field/result... skipping.
------------
Retrieving Results for Index 7: Karlovasi.
Closest hotel in Karlovasi is Samian Mare Hotel Suites & Spa.
------------
Retrieving Results for Index 8: Vila Franca do Campo.
Closest hotel in Vila Franca do Campo is Vinha d'Areia Beach Hotel.
------------
Retrieving

Closest hotel in Hithadhoo is Pebbles Inn.
------------
Retrieving Results for Index 78: Sohag.
Closest hotel in Sohag is برج 25 يناير.
------------
Retrieving Results for Index 79: Tegul'det.
Missing field/result... skipping.
------------
Retrieving Results for Index 80: Jumla.
Closest hotel in Jumla is Hotel Chandra.
------------
Retrieving Results for Index 81: Hambantota.
Closest hotel in Hambantota is The Peacock Beach Hotel Hambantota.
------------
Retrieving Results for Index 82: Chui.
Closest hotel in Chui is Hotel Internacional.
------------
Retrieving Results for Index 83: Saskylakh.
Missing field/result... skipping.
------------
Retrieving Results for Index 84: Birjand.
Closest hotel in Birjand is مهمان پذیر نوید.
------------
Retrieving Results for Index 85: San Patricio.
Closest hotel in San Patricio is Americas Best Value Inn Sinton.
------------
Retrieving Results for Index 86: Beloha.
Missing field/result... skipping.
------------
Retrieving Results for Index 87: Palu.


Closest hotel in Nizhniy Odes is Gostinitsa Uyut.
------------
Retrieving Results for Index 156: Batamshinskiy.
Missing field/result... skipping.
------------
Retrieving Results for Index 157: Port Augusta.
Closest hotel in Port Augusta is Comfort Inn & Suites Augusta Westside.
------------
Retrieving Results for Index 158: Røros.
Closest hotel in Røros is Bergstadens Hotel.
------------
Retrieving Results for Index 159: Xichang.
Closest hotel in Xichang is Holiday Inn Express Xichang City Center.
------------
Retrieving Results for Index 160: Necochea.
Closest hotel in Necochea is Hotel Firenze.
------------
Retrieving Results for Index 161: Cabo San Lucas.
Closest hotel in Cabo San Lucas is Medano Hotel and Suites.
------------
Retrieving Results for Index 162: Saint-Augustin.
Missing field/result... skipping.
------------
Retrieving Results for Index 163: Castro.
Closest hotel in Castro is Central Palace Hotel.
------------
Retrieving Results for Index 164: Rawson.
Closest hotel in 

Closest hotel in Vardø is Vardø Hotel AS.
------------
Retrieving Results for Index 233: Bethanien.
Closest hotel in Bethanien is Bethanie Hotel.
------------
Retrieving Results for Index 234: San Jose.
Closest hotel in San Jose is DoubleTree by Hilton Hotel San Jose.
------------
Retrieving Results for Index 235: Tautira.
Missing field/result... skipping.
------------
Retrieving Results for Index 236: Lorengau.
Closest hotel in Lorengau is Lorengau Harbourside Hotel.
------------
Retrieving Results for Index 237: Tolmachëvo.
Closest hotel in Tolmachëvo is Skyport Hotel.
------------
Retrieving Results for Index 238: Aykhal.
Missing field/result... skipping.
------------
Retrieving Results for Index 239: Garoowe.
Closest hotel in Garoowe is Martisoor Hotel.
------------
Retrieving Results for Index 240: Belyy Yar.
Missing field/result... skipping.
------------
Retrieving Results for Index 241: Jacmel.
Closest hotel in Jacmel is Cap Lamandou Hotel.
------------
Retrieving Results for In

Missing field/result... skipping.
------------
Retrieving Results for Index 311: Arman'.
Missing field/result... skipping.
------------
Retrieving Results for Index 312: Hīt.
Missing field/result... skipping.
------------
Retrieving Results for Index 313: Cobija.
Closest hotel in Cobija is Brasiléia Palace Hotel.
------------
Retrieving Results for Index 314: Pacifica.
Closest hotel in Pacifica is Pacifica Beach Hotel.
------------
Retrieving Results for Index 315: Manaure.
Closest hotel in Manaure is HOTEL BRISAS DEL MAR MANAURE.
------------
Retrieving Results for Index 316: Kaitangata.
Missing field/result... skipping.
------------
Retrieving Results for Index 317: Banyo.
Closest hotel in Banyo is Hôtel Municipal de Banyo.
------------
Retrieving Results for Index 318: Verkhoyansk.
Missing field/result... skipping.
------------
Retrieving Results for Index 319: Ahipara.
Closest hotel in Ahipara is Ahipara Bay Motel.
------------
Retrieving Results for Index 320: Alofi.
Closest hotel

Closest hotel in Narsaq is Narsaq Kayak Hostel.
------------
Retrieving Results for Index 391: La Ronge.
Closest hotel in La Ronge is The Harbour Inn.
------------
Retrieving Results for Index 392: Tabou.
Closest hotel in Tabou is Hotel Les Rochers.
------------
Retrieving Results for Index 393: Escanaba.
Closest hotel in Escanaba is Comfort Suites.
------------
Retrieving Results for Index 394: Taltal.
Closest hotel in Taltal is Hotel Mi Tampi.
------------
Retrieving Results for Index 395: Anito.
Closest hotel in Anito is The Yellow House.
------------
Retrieving Results for Index 396: Hay River.
Closest hotel in Hay River is Ptarmigan Inn.
------------
Retrieving Results for Index 397: Teya.
Missing field/result... skipping.
------------
Retrieving Results for Index 398: Sfântu Gheorghe.
Closest hotel in Sfântu Gheorghe is Isabella Boutique Hotel.
------------
Retrieving Results for Index 399: Zhigansk.
Missing field/result... skipping.
------------
Retrieving Results for Index 400:

Closest hotel in Anloga is Pin Drop Hotel.
------------
Retrieving Results for Index 470: Grindavik.
Closest hotel in Grindavik is The Retreat at Blue Lagoon Iceland.
------------
Retrieving Results for Index 471: Laguna.
Closest hotel in Laguna is Hampton Inn & Suites Sacramento-Elk Grove Laguna I-5.
------------
Retrieving Results for Index 472: Coahuayana Viejo.
Closest hotel in Coahuayana Viejo is Hotel Los Arcos.
------------
Retrieving Results for Index 473: Spasskoye.
Closest hotel in Spasskoye is Gostinitsa "Voyazh".
------------
Retrieving Results for Index 474: Salinópolis.
Closest hotel in Salinópolis is Hotel Privê do Atalaia.
------------
Retrieving Results for Index 475: Atar.
Closest hotel in Atar is les toiles maures.
------------
Retrieving Results for Index 476: Huanren.
Closest hotel in Huanren is Liangyu Hotel.
------------
Retrieving Results for Index 477: Rozhyshche.
Missing field/result... skipping.
------------
Retrieving Results for Index 478: Rodrigues Alves.


Missing field/result... skipping.
------------
Retrieving Results for Index 548: Linqiong.
Closest hotel in Linqiong is Lanting Hotel.
------------
Retrieving Results for Index 549: Koumac.
Closest hotel in Koumac is Karem Bay.
------------
Retrieving Results for Index 550: Dubinino.
Missing field/result... skipping.
------------
Retrieving Results for Index 551: Pascagoula.
Closest hotel in Pascagoula is Hilton Garden Inn Pascagoula.
------------
Retrieving Results for Index 552: Kabo.
Missing field/result... skipping.
------------
Retrieving Results for Index 553: Statesboro.
Closest hotel in Statesboro is Holiday Inn Statesboro-University Area, an IHG Hotel.
------------
Retrieving Results for Index 554: Maniitsoq.
Closest hotel in Maniitsoq is Hotel Heilmann Lyberth.
------------
Retrieving Results for Index 555: Aktau.
Closest hotel in Aktau is Caspian Riviera Grand Palace Hotel.
------------
Retrieving Results for Index 556: Neyshabur.
Closest hotel in Neyshabur is Amiran Hotel &

In [27]:
# Display the hotel dataframe
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
0,Hermanus,ZA,-34.4187,19.2345,Misty Waves Boutique Hotel
1,Clifton Springs,AU,-38.1500,144.5667,Drysdale Hotel
2,At-Bashi,KG,41.1702,75.8106,Hostel At-Bashy
3,Kruisfontein,ZA,-34.0033,24.7314,Humansdorp Boutique Hotel
4,Atuona,PF,-9.8000,-139.0333,Hôtel Hiva Oa Hanakee Lodge
...,...,...,...,...,...
564,Gushikawa,JP,26.3544,127.8686,Hotel R9 The Yard
565,Crotone,IT,39.0851,17.1178,Hotel Ristorante Lido degli Scogli
566,Byron Bay,AU,-28.6500,153.6167,"Beach Hotel Resort, Byron Bay"
567,Pacific Grove,US,36.6177,-121.9166,Hotel 1110


In [25]:
# NOTE: Do not change any of the code in this cell
# Analyze the code below for variables but these should be all columns in hotel_df

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

KeyError: "None of [Index(['Lat', 'Lng'], dtype='object')] are in the [columns]"

In [26]:
# Add marker layer and info box content ontop of heat map to provide hotel information
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))